In [ ]:
!pip install cherche

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.4/385.4 KB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 68.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64

In [ ]:
from cherche import retrieve

In [ ]:

import pandas as pd

from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
caminho_query = "/content/gdrive/MyDrive/Colab Notebooks/dados-conle-anonimizado-assunto-notnull (1).csv"

Mounted at /content/gdrive


# Base de dados câmara dos deputados

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/proposicao-tema-completo.csv")

In [ ]:
df_assunto= pd.read_csv(caminho_query, encoding='utf-8', delimiter=";")

In [ ]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [ ]:
def verificar(y,top_n):
    for d in top_n:
      if y == df.loc[d["id"],"txtNome"]:
        return 1
    return 0

In [ ]:
def avaliacaoRecall(isPreprocess):

    quant_encontrado=0
    quant_relevante =0
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        top_n = retriever(tokenized_query3)
        #top_n = [df.loc[d["id"],"name"] for d in top_n_stem_l]

        
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    print("R@20: "+str(recall))


# Pré processamento

## 1- Sem pré processamento

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":str(df.loc[i,"imgArquivoTeorPDF"])
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"], documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(False)

In [ ]:
documentos = None
retriever =None

## 2- Letra mínuscula

In [ ]:
def preprocess(txt):
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = " ".join(terms)
    return terms

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]




In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"], documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.4745762711864407


In [ ]:
documentos = None
retriever =None

## 3- Letra mínuscula + remoção de pontuação

In [ ]:
def preprocess(txt):
    stopwords = list(punctuation)
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]





In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"], documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.4745762711864407


In [ ]:
documentos = None
retriever =None

## 4- Letra mínuscula + remoção de pontuação e remoção de acentuação

In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = list(punctuation)
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]






In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"], documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.5457627118644067


In [ ]:
documentos = None
retriever =None

## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [ ]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [ ]:
documentos = [
    {
        "id":i,
        "name":str(df.loc[i,"txtNome"]).strip(),
        "text":preprocess(str(df.loc[i,"imgArquivoTeorPDF"]))
    }

   for i in range(len(df))
    ]






In [ ]:
retriever = retrieve.BM25Okapi(key="id", on=["text"], documents=documentos,k=20,k1=1.5,b=0.75,epsilon=0.25)

Recall

In [ ]:
avaliacaoRecall(True)

R@20: 0.5661016949152542


In [ ]:
documentos = None
retriever =None